https://matheuscammarosanohidalgo.medium.com/a-very-simple-variational-quantum-classifier-vqc-64e8ec26589d
tutaj dodają jakieś dane

In [2]:
from modules.Ansatz import circuit
import pennylane.numpy as np

def variational_classifier(weights, bias, x, num_qubits, state_prep):
    return circuit(weights, x, num_qubits, state_prep) + bias


def square_loss(labels, predictions):
    # We use a call to qml.math.stack to allow subtracting the arrays directly
    return np.mean((labels - qml.math.stack(predictions)) ** 2)


def cost(weights, bias, X, Y, num_qubits, state_prep):
    predictions = [variational_classifier(weights, bias, x, num_qubits, state_prep) for x in X]
    return square_loss(Y, predictions)

Y_train: Values: [-1.  1.] Counts: [40 40]
Y_val: Values: [-1.  1.] Counts: [4 6]
Y_test: Values: [-1.  1.] Counts: [6 4]


ValueError: Features must be of length 16; got length 4. Use the 'pad_with' argument for automated padding.

In [2]:
from ucimlrepo import fetch_ucirepo
# fetch dataset
ionosphere = fetch_ucirepo(id=52)
# data (as pandas dataframes)
X = ionosphere.data.features
y = ionosphere.data.targets

Weights: [[[0.5488135  0.71518937 0.60276338]
  [0.54488318 0.4236548  0.64589411]
  [0.43758721 0.891773   0.96366276]
  [0.38344152 0.79172504 0.52889492]]

 [[0.56804456 0.92559664 0.07103606]
  [0.0871293  0.0202184  0.83261985]
  [0.77815675 0.87001215 0.97861834]
  [0.79915856 0.46147936 0.78052918]]]
Bias:  0.1
Iter:    1 | Cost: 9.1710255 | Accuracy: 0.5937500
Iter:    2 | Cost: 5669.6905856 | Accuracy: 0.5625000
Iter:    3 | Cost: 4771629.1461842 | Accuracy: 0.3125000
Iter:    4 | Cost: 3999450527.9809947 | Accuracy: 0.5625000
Iter:    5 | Cost: 3355704034895.1791992 | Accuracy: 0.4062500
Iter:    6 | Cost: 2815544254929149.0000000 | Accuracy: 0.5937500
Iter:    7 | Cost: 2362333996061953536.0000000 | Accuracy: 0.6562500
Iter:    8 | Cost: 1982075675061019213824.0000000 | Accuracy: 0.5625000
Iter:    9 | Cost: 1663026474962005144895488.0000000 | Accuracy: 0.6562500
Iter:   10 | Cost: 1395333735821491778288091136.0000000 | Accuracy: 0.5937500
Iter:   11 | Cost: 1170730751213156

In [ ]:
X_dropped = X.drop(X.columns[[0, 1]], axis=1)
print('Shape:',X_dropped.shape[1])
y_change = y.replace({'g': 1, 'b': -1})
# Normalize each row in X
# Convert DataFrame to numpy array
X_array = X_dropped.to_numpy()
# Calculate the normalization factor for each row
normalization = np.sqrt(np.sum(X_array**2, axis=-1))
X_norm_array = (X_array.T / normalization).T
X_norm_array
Y_train = y.to_numpy()
from sklearn.model_selection import train_test_split
# First, split into training + validation and test
X_train, X_test, Y_train, Y_test = train_test_split(
    X_norm_array, Y_train, test_size=0.2, shuffle=True, random_state=42)
arrays = [Y_train, Y_test]
names = ["Y_train", "Y_test"]

for y, name in zip(arrays, names):
    values, counts = np.unique(y, return_counts=True)
    print(f"{name}: Values: {values} Counts: {counts}")

In [ ]:
from modules.Ansatz import circuit
from modules.metrics import accuracy
import pennylane as qml
import pennylane.numpy as np
X_val=X_test
Y_val=Y_test
num_qubits = 5
num_layers = 6

weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)
learning_rate = 0.01
batch_size = 5
num_epochs = 60
state_prep='Mottonen'
# train the variational classifier
weights = weights_init
bias = bias_init

# params = circuit_training(X_train,Y_train,X_test, Y_test,num_qubits, num_layers,bias_init,learning_rate,batch_size, num_epochs,state_prep='Mottonen',seed = 0)
all_accuracy = []

opt = qml.NesterovMomentumOptimizer(stepsize=learning_rate)

for epoch in range(num_epochs):
    # Update the weights by one optimizer step, using only a limited batch of data
    batch_index = np.random.randint(0, len(X_train), (batch_size,))
    X_batch = X_train[batch_index]
    Y_batch = Y_train[batch_index]
    weights, bias = opt.step(cost, weights, bias, X=X_batch, Y=Y_batch, num_qubits=num_qubits, state_prep=state_prep)

    # Compute predictions on train and validation set
    predictions_train = [np.sign(variational_classifier(weights, bias, x, num_qubits, state_prep)) for x in X_batch]

    predictions_val = [np.sign(variational_classifier(weights, bias, x, num_qubits, state_prep)) for x in X_val]
    current_cost = cost(weights, bias, X_batch, Y_batch, num_qubits, state_prep)

    # Compute accuracy on train and validation set
    print(Y_batch[0],'   ',predictions_train[0])
    acc_train = accuracy(Y_batch, predictions_train)
    acc_val = accuracy(Y_val, predictions_val)

    print(f"Epoch: {epoch} | Cost: {current_cost:0.7f} | "f"Acc train: {acc_train:0.7f} | Acc validation: {acc_val:0.7f}"
          )